In [1]:
%matplotlib auto
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import math
from mpl_toolkits.mplot3d import Axes3D

Using matplotlib backend: TkAgg


### NTU60的动作

In [2]:
action_names = ['drink water 1', 'eat meal/snack 2', 'brushing teeth 3', 'brushing hair 4', 'drop 5', 'pickup 6',
            'throw 7', 'sitting down 8', 'standing up 9', 'clapping 10', 'reading 11', 'writing 12',
            'tear up paper 13', 'wear jacket 14', 'take off jacket 15', 'wear a shoe 16', 'take off a shoe 17',
            'wear on glasses 18','take off glasses 19', 'put on a hat/cap 20', 'take off a hat/cap 21', 'cheer up 22',
            'hand waving 23', 'kicking something 24', 'put/take out sth 25', 'hopping 26', 'jump up 27',
            'make a phone call 28', 'playing with a phone 29', 'typing on a keyboard 30',
            'pointing to sth with finger 31', 'taking a selfie 32', 'check time (from watch) 33',
            'rub two hands together 34', 'nod head/bow 35', 'shake head 36', 'wipe face 37', 'salute 38',
            'put the palms together 39', 'cross hands in front 40', 'sneeze/cough 41', 'staggering 42', 'falling 43',
            'touch head 44', 'touch chest 45', 'touch back 46', 'touch neck 47', 'nausea or vomiting condition 48',
            'use a fan 49', 'punching 50', 'kicking other person 51', 'pushing other person 52',
            'pat on back of other person 53', 'point finger at the other person 54', 'hugging other person 55',
            'giving sth to other person 56', 'touch other person pocket 57', 'handshaking 58',
            'walking towards each other 59', 'walking apart from each other 60']
max_frame = 300
max_body = 2
max_joint = 25

### 读取数据

#### 1.读取skeleton文件

In [3]:
def read_skeleton(file):
    with open(file, 'r') as f:
        skeleton_sequence = {}
        skeleton_sequence['numFrame'] = int(f.readline())
        skeleton_sequence['frameInfo'] = []
        for t in range(skeleton_sequence['numFrame']):
            frame_info = {}
            frame_info['numBody'] = int(f.readline())
            frame_info['bodyInfo'] = []
            for m in range(frame_info['numBody']):
                body_info = {}
                body_info_key = [
                    'bodyID', 'clipedEdges', 'handLeftConfidence',
                    'handLeftState', 'handRightConfidence', 'handRightState',
                    'isResticted', 'leanX', 'leanY', 'trackingState'
                ]
                body_info = {
                    k: float(v)
                    for k, v in zip(body_info_key, f.readline().split())
                }
                body_info['numJoint'] = int(f.readline())
                body_info['jointInfo'] = []
                for v in range(body_info['numJoint']):
                    joint_info_key = [
                        'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                        'orientationW', 'orientationX', 'orientationY',
                        'orientationZ', 'trackingState'
                    ]
                    joint_info = {
                        k: float(v)
                        for k, v in zip(joint_info_key, f.readline().split())
                    }
                    body_info['jointInfo'].append(joint_info)
                frame_info['bodyInfo'].append(body_info)
            skeleton_sequence['frameInfo'].append(frame_info)
    return skeleton_sequence

#### 2.读取60个样本，每个样本归属于一类动作，共60类

In [6]:
data_file = "data/c003"
data = np.zeros((60, 3,max_frame, max_joint, max_body))   # 60个不同动作的样本坐标
frame_nums = np.zeros((60))
for file in os.listdir(data_file):
    action_class = int(file[file.find('A')+1:file.find('A')+4])   # 动作编号
    action_name = action_names[action_class-1]    # 动作名称
    print(action_name)
    file_path = os.path.join(data_file, file)    # skeleton文件路径
    skeleton_info = read_skeleton(file_path)   # 读取一个skeleton文件信息
    frame_nums[action_class-1] = skeleton_info['numFrame']
    for n, f in enumerate(skeleton_info['frameInfo']):
        for m, b in enumerate(f['bodyInfo']):
            for j, v in enumerate(b['jointInfo']):
                if m < max_body and j < max_joint:
                    data[action_class-1, :, n, j, m] = [v['x'], v['y'], v['z']]
                else:
                    pass

drink water 1
eat meal/snack 2
brushing teeth 3
brushing hair 4
drop 5
pickup 6
throw 7
sitting down 8
standing up 9
clapping 10
reading 11
writing 12
tear up paper 13
wear jacket 14
take off jacket 15
wear a shoe 16
take off a shoe 17
wear on glasses 18
take off glasses 19
put on a hat/cap 20
take off a hat/cap 21
cheer up 22
hand waving 23
kicking something 24
put/take out sth 25
hopping 26
jump up 27
make a phone call 28
playing with a phone 29
typing on a keyboard 30
pointing to sth with finger 31
taking a selfie 32
check time (from watch) 33
rub two hands together 34
nod head/bow 35
shake head 36
wipe face 37
salute 38
put the palms together 39
cross hands in front 40
sneeze/cough 41
staggering 42
falling 43
touch head 44
touch chest 45
touch back 46
touch neck 47
nausea or vomiting condition 48
use a fan 49
punching 50
kicking other person 51
pushing other person 52
pat on back of other person 53
point finger at the other person 54
hugging other person 55
giving sth to other pers

In [7]:
print(data.shape)
print(frame_nums)

(60, 3, 300, 25, 2)
[ 72.  95.  95. 102.  71.  83.  77.  79.  60.  71. 151. 120. 113. 159.
 175. 156. 124.  90.  70.  80.  81.  72.  84.  72.  85. 104.  71. 122.
 109. 134.  63.  87.  66.  72.  71.  52. 107.  73.  53.  53.  65. 112.
  82.  84. 102.  80.  95.  80.  80.  64.  67.  68.  73.  58.  76.  81.
 100.  67. 104.  65.]


### 开始绘制骨骼图

#### 1.归一化骨骼

In [8]:
def normal_skeleton(data, frame_num):
    #  use as center joint
    data = np.transpose(data,(3,1,2,0))
    data = data[:,0:frame_num,:,:]
    center_joint = data[0, :, 0, :]
    
    center_jointx = np.mean(center_joint[:,0])
    center_jointy = np.mean(center_joint[:,1])
    center_jointz = np.mean(center_joint[:,2])
    center = np.array([center_jointx, center_jointy, center_jointz])
    data = data - center
    data = np.transpose(data,(3,1,2,0))
    return data

#### 2.旋转操作

In [9]:
def rotation(data, angle):
    # 旋转数据操作
    # ntu沿y轴
    rotation_matrix_y = np.array([[math.cos(angle), 0, -math.sin(angle)],
                                  [0, 1, 0],
                                  [math.sin(angle), 0, math.cos(angle)]])
    data = np.dot(rotation_matrix_y,data)
    return data

#### 3.绘制旋转骨骼图

In [38]:
k_index = 11
np.random.seed(202296)
skeleton_data = data[k_index]  # 取第k_index个骨骼动作信息
# 归一化操作
skeleton_data = normal_skeleton(skeleton_data, int(frame_nums[k_index]))
C, T, V, M = skeleton_data.shape
# 骨骼连接关系
connecting_joint = np.array([2,1,21,3,21,5,6,7,21,9,10,11,1,13,14,15,1,17,18,19,2,23,8,25,12])
# 绘制画布
fig = plt.figure(dpi=300)
ax = Axes3D(fig)
ax.view_init(0, 0)
# 绘制3d骨骼点的每一帧
# plt.cla()   # 清除当前画布的axes
# ax.set_xlim3d([-0.8,0.8])
# ax.set_ylim3d([-0.8,0.8])
# ax.set_zlim3d([-0.8,0.8])
# 绘制第55帧里的第一个人
for f in range(int(frame_nums[k_index])):
    plt.cla()   # 清除当前画布的axes
    ax.set_xlim3d([-0.8,0.8])
    ax.set_ylim3d([-0.8,0.8])
    ax.set_zlim3d([-0.8,0.8])
    skeleton_data1 = skeleton_data[:,f,:,0]
    x = skeleton_data1[0]
    y = skeleton_data1[1]
    z = skeleton_data1[2]
    # skeleton_data_rotation = rotation(skeleton_data, math.pi/3*3)
    # x_rotation = skeleton_data_rotation[0]
    # y_rotation = skeleton_data_rotation[1]
    # z_rotation = skeleton_data_rotation[2]
    for v in range(V):
        col = (np.random.random(), np.random.random(), np.random.random())
        k = connecting_joint[v]-1
        ax.plot([z[v],z[k]],[x[v],x[k]],[y[v],y[k]], color=col, marker='o')   # 旋转前
    #     ax.plot([z_rotation[v],z_rotation[k]],[x_rotation[v],x_rotation[k]],[y_rotation[v],y_rotation[k]], color=col, marker='o')   # 旋转后
#     ax.set_xlabel('X')
#     ax.set_ylabel('Y')
#     ax.set_zlabel('Z')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.axis('off')
    plt.savefig("pic/writing/{}.png".format(f),dpi=300)
# plt.show()

#### 4.绘制空间遮挡

###### 4.1 对单一样本进行旋转操作

In [8]:
def rotation_xyz(data, angle):
    # 旋转数据操作
    # ntu沿y轴
    rotation_matrix_y = np.array([[math.cos(angle), 0, -math.sin(angle)],
                                  [0, 1, 0],
                                  [math.sin(angle), 0, math.cos(angle)]])
    c,j,m = data.shape
    data = data.reshape((c,j*m))
    data = np.dot(rotation_matrix_y,data)
    data = data.reshape((c,j,m))
    return data

##### 4.2对旋转后的样本骨骼范围进行测算

In [9]:
def skel_boundary(data):
    maxX, minX = data[0, :, 0].max(), data[0, :, 0].min()
    maxY, minY = data[1, :, 0].max(), data[1, :, 0].min()
    return maxX, minX, maxY, minY

###### 4.3判断是否满足四边形条件

In [10]:
def judge_quadrilateral(coordinate, occ1, occ2):
    world_coordinate = np.array(coordinate)
    # 遮挡操作
    occ1_maxX, occ1_minX, occ1_maxY, occ1_minY = occ1
    occ2_maxX, occ2_minX, occ2_maxY, occ2_minY = occ2
    occ1_vertex_a = np.array([occ1_minX, occ1_maxY, 2.6])
    occ1_vertex_b = np.array([occ1_maxX, occ1_maxY, 2.6])
    occ1_vertex_c = np.array([occ1_maxX, occ1_minY, 2.6])
    occ1_vertex_d = np.array([occ1_minX, occ1_minY, 2.6])
    z1 = np.dot((occ1_vertex_b - occ1_vertex_a), (world_coordinate - occ1_vertex_a))
    z2 = np.dot((occ1_vertex_c - occ1_vertex_b), (world_coordinate - occ1_vertex_b))
    z3 = np.dot((occ1_vertex_d - occ1_vertex_c), (world_coordinate - occ1_vertex_c))
    z4 = np.dot((occ1_vertex_a - occ1_vertex_d), (world_coordinate - occ1_vertex_d))

    occ2_vertex_a = np.array([occ2_minX, occ2_maxY, 2.6])
    occ2_vertex_b = np.array([occ2_maxX, occ2_maxY, 2.6])
    occ2_vertex_c = np.array([occ2_maxX, occ2_minY, 2.6])
    occ2_vertex_d = np.array([occ2_minX, occ2_minY, 2.6])
    z5 = np.dot((occ2_vertex_b - occ2_vertex_a), (world_coordinate - occ2_vertex_a))
    z6 = np.dot((occ2_vertex_c - occ2_vertex_b), (world_coordinate - occ2_vertex_b))
    z7 = np.dot((occ2_vertex_d - occ2_vertex_c), (world_coordinate - occ2_vertex_c))
    z8 = np.dot((occ2_vertex_a - occ2_vertex_d), (world_coordinate - occ2_vertex_d))
    is_positive1 = (z1 * z2 > 0) and (z3 * z4 > 0) and (z1 * z3 > 0)
    is_positive2 = (z5 * z6 > 0) and (z7 * z8 > 0) and (z5 * z7 > 0)
    if is_positive1 or is_positive2:
        image_coordinate = np.array([0, 0, 0])
    else:
        image_coordinate = world_coordinate
    return image_coordinate

##### 4.4 进行遮挡操作

In [11]:
def occlusion(data, angles):
    # 沿y轴旋转
    data_left = rotation_xyz(data, angles[0])
    data_right = rotation_xyz(data, angles[1])
    # 寻找三个视角骨骼的边界范围
    maxX1, minX1, maxY1, minY1 = skel_boundary(data_left)
    maxX2, minX2, maxY2, minY2 = skel_boundary(data)
    maxX3, minX3, maxY3, minY3 = skel_boundary(data_right)
    # 左边视角取骨骼右半部分1/2,中间视角取骨骼左半部分1/3，右半部分1/3,右边视角取骨骼左半部分1/2
    occ1_minX, occ1_maxX = maxX1 - (maxX1 - minX1) / 2, minX2 + (maxX2 - minX2) /3
    occ1_minY, occ1_maxY = min(minY1, minY2), max(maxY1, maxY2)
    occ2_minX, occ2_maxX = maxX2 - (maxX2 - minX2) /3, minX3 + (maxX3 - minX3) / 2
    occ2_minY, occ2_maxY = min(minY2, minY3), max(maxY2, maxY3)
    occ1 = (occ1_maxX, occ1_minX, occ1_maxY, occ1_minY)
    occ2 = (occ2_maxX, occ2_minX, occ2_maxY, occ2_minY)
    # 将遮挡范围内的骨骼点置为0
    for m in range(2):
        for j in range(25):
            data_left[:,j,m] = judge_quadrilateral(data_left[:,j,m], occ1, occ2)
            data[:, j, m] = judge_quadrilateral(data[:, j, m], occ1, occ2)
            data_right[:, j, m] = judge_quadrilateral(data_right[:, j, m], occ1, occ2)
    return (data_left, data, data_right),(occ1,occ2)

##### 4.5 处理数据

In [12]:
class_index = 8
angles = [-math.pi/4,math.pi/4]
skeleton = data[class_index]
# 构建当前文件的遮挡骨骼坐标
skeleton_data = np.zeros((3,int(frame_nums[class_index]),25,2))
skeleton_data_left = np.zeros((3,int(frame_nums[class_index]),25,2))
skeleton_data_right = np.zeros((3,int(frame_nums[class_index]),25,2))
# 两个遮挡区域
occ_area = np.zeros((2,300,4))
for t in range(int(frame_nums[class_index])):
    data_coordinate, occ = occlusion(skeleton[:,t,:,:], angles)
    frame_data_left, frame_data, frame_data_right = data_coordinate
    skeleton_data_left[:,t,:,:] = frame_data_left
    skeleton_data_right[:,t,:,:] = frame_data_right
    skeleton_data[:,t,:,:] = frame_data
    occ_area[:,t,:] = occ

##### 4.6绘制遮挡区域

In [13]:
np.random.seed(202296)
# 归一化操作
# skeleton_data = normal_skeleton(skeleton_data, int(frame_nums[k_index]))
# 骨骼连接关系
connecting_joint = np.array([2,1,21,3,21,5,6,7,21,9,10,11,1,13,14,15,1,17,18,19,2,23,8,25,12])
# 绘制画布
fig = plt.figure()
ax = Axes3D(fig)
# ax.view_init(30, 45)
# 绘制3d骨骼点的每一帧
plt.cla()   # 清除当前画布的axes
# ax.set_xlim3d([-0.8,0.8])
# ax.set_ylim3d([-0.8,0.8])
# ax.set_zlim3d([-0.8,0.8])


# 绘制遮挡区域
occ1_x = np.arange(occ_area[0,55,1],occ_area[0,55,0],0.1)
print("occ1_x:",occ_area[:,55,:])
occ1_y = np.arange(occ_area[0,55,3],occ_area[0,55,2],0.1)
occ1_x, occ1_y = np.meshgrid(occ1_x, occ1_y)
print("occ1_x.shape:",occ1_x)
# ax.plot_surface()
# plt.Rectangle((occ_area[0,55,1], occ_area[0,55,2]), occ_area[0,55,0]-occ_area[0,55,1], occ_area[0,55,2]-occ_area[0,55,3], 
#               fill=True, edgecolor = 'red',linewidth=1)


# 绘制第55帧里的第一个人
# 原骨骼数据
skeleton_data = skeleton_data[:,55,:,0]
x = skeleton_data[0]
y = skeleton_data[1]
z = skeleton_data[2]
# 左边骨骼数据
skeleton_data_left = skeleton_data_left[:,55,:,0]
x_left = skeleton_data_left[0]
y_left = skeleton_data_left[1]
z_left = skeleton_data_left[2]
# 右边骨骼数据
skeleton_data_right = skeleton_data_right[:,55,:,0]
x_right = skeleton_data_right[0]
y_right = skeleton_data_right[1]
z_right = skeleton_data_right[2]
print("skeleton_data:",skeleton_data)
print("skeleton_data_left:",skeleton_data_left)
print("skeleton_data_right:",skeleton_data_right)
# skeleton_data_rotation = rotation(skeleton_data, math.pi/3*3)
# x_rotation = skeleton_data_rotation[0]
# y_rotation = skeleton_data_rotation[1]
# z_rotation = skeleton_data_rotation[2]
for v in range(25):
    col = (np.random.random(), np.random.random(), np.random.random())
    k = connecting_joint[v]-1
    ax.plot([z[v],z[k]],[x[v],x[k]],[y[v],y[k]], color=col, marker='o')   # 原骨骼
    ax.plot([z_left[v],z_left[k]],[x_left[v],x_left[k]],[y_left[v],y_left[k]], color=col, marker='o')   # 左侧骨骼
    ax.plot([z_right[v],z_right[k]],[x_right[v],x_right[k]],[y_right[v],y_right[k]], color=col, marker='o')   # 右侧骨骼
ax.set_xlabel('z')
ax.set_ylabel('x')
ax.set_zlabel('y')
# ax.set_xticks([])
# ax.set_yticks([])
# ax.set_zticks([])
ax.axis('on')
# plt.savefig("pic/rotation180Class9.svg",dpi=300)
plt.show(block=True)

C:\Users\admin\AppData\Local\Temp/ipykernel_14184/3717162589.py:8: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = Axes3D(fig)


occ1_x: [[ 0.0757947   2.32079365  1.050069   -0.5660943 ]
 [-2.06137435  0.24739305  1.050069   -0.5660943 ]]
occ1_x.shape: []
skeleton_data: [[ 0.         0.         0.         0.2119002  0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.3526803  0.         0.         0.         0.
   0.       ]
 [ 0.         0.         0.         1.050069   0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.        -0.5660943  0.         0.         0.         0.
   0.       ]
 [ 0.         0.         0.         3.023713   0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         3.18162    0.         0.         0.         0.
   0.       ]]
skeleton_data_left: [[ 2.33653451  0.          0.          2.28792404  0.         